In [1]:
import requests
import base64
from urllib.parse import urlencode
import json
import datetime

#import constants from KEYS module. create a module called KEYS.py and Add your own Client ID and Client Secret to it
import KEYS

In [2]:
requests.__version__

'2.25.1'

In [3]:
#assign keys to values here

client_id = KEYS.CLIENT_ID
client_secret = KEYS.CLIENT_SECRET

In [4]:
#format and encode credential in to expected form for api

client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())
print(base64.b64decode(client_creds_b64).decode())
print(client_creds)
print(client_creds_b64.decode())

af37f76b6c204b16b5f781ce1e54603d:cdc8546a2bdd42ea96f3fe6a860ac1d3
af37f76b6c204b16b5f781ce1e54603d:cdc8546a2bdd42ea96f3fe6a860ac1d3
YWYzN2Y3NmI2YzIwNGIxNmI1Zjc4MWNlMWU1NDYwM2Q6Y2RjODU0NmEyYmRkNDJlYTk2ZjNmZTZhODYwYWMxZDM=


In [5]:
#Builds the parts of the querey
token_url = "https://accounts.spotify.com/api/token"
method = "POST"
token_data = {
    "grant_type":"client_credentials",
}
token_headers = {
    "Authorization" : f"Basic {client_creds_b64.decode()}",
}

In [6]:
#makes GET request to spotify API
r = requests.post(token_url, headers=token_headers, data=token_data)

#validates result, then prints
valid_request = r.status_code in range(200,299)
print(r.json())

{'access_token': 'BQAUksvwPG6qjkplWLFhgoSiOtrW5k7QnSS5iK_0K0PH8BMGkYykQLpIETuVBZZHbcTp30K_DtAeL_0Dsa8', 'token_type': 'Bearer', 'expires_in': 3600}


In [7]:
#uses validation boolean to decide if to parse
if valid_request:
    token_response_data = r.json() #convert json to dict
    now = datetime.datetime.now() # create datetime obj for now
    access_token = token_response_data['access_token'] #retrieve token from response
    expires_in = token_response_data['expires_in'] #retrieve token lifetime
    expires = now + datetime.timedelta(seconds=expires_in) #creates datetime for when token expires
    did_expire = expires < datetime.datetime.now() #boolean to rep if expired

In [8]:
did_expire

False

In [9]:
expires

datetime.datetime(2021, 5, 5, 16, 46, 38, 953307)

# Base API Class 

In [19]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentiatls(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentiatls()
        return  {
            "Authorization" : f"Basic {client_creds_b64}",
        }
    
    def get_token_data(self):
        return {
            "grant_type":"client_credentials",
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_headers = self.get_token_headers()
        token_data = self.get_token_data()
        r = requests.post(token_url, headers=token_headers, data=token_data)
        if r.status_code not in range(200,299):
            return False
        data = r.json() #convert json to dict
        now = datetime.datetime.now() # create datetime obj for now
        access_token = token_response_data['access_token'] #retrieve token from response
        expires_in = token_response_data['expires_in'] #retrieve token lifetime
        expires = now + datetime.timedelta(seconds=expires_in) #creates datetime for when token expires
        self.access_token = access_token
        self.access_token_expiers = expires
        self.access_token_did_expire = expires < datetime.datetime.now() #boolean to rep if expired
        return True        

In [20]:
client = SpotifyAPI(client_id, client_secret)

In [21]:
client.perform_auth()

True

In [22]:
client.access_token

'BQAUksvwPG6qjkplWLFhgoSiOtrW5k7QnSS5iK_0K0PH8BMGkYykQLpIETuVBZZHbcTp30K_DtAeL_0Dsa8'

In [ ]:
# spotify.search

In [23]:
access_token = client.access_token

In [28]:
headers = {
    "Authorization": f"Bearer {access_token}"
}
params = {
    "q":"ONLY Zhu",
    "type":"track"
}
endpoint = "https://api.spotify.com/v1/search"

r = requests.get(endpoint, params=params, headers=headers)

In [29]:
r.status_code

200

In [32]:
r.json()

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=ONLY+Zhu&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/28j8lBWDdDSHSSt5oPlsX2'},
       'href': 'https://api.spotify.com/v1/artists/28j8lBWDdDSHSSt5oPlsX2',
       'id': '28j8lBWDdDSHSSt5oPlsX2',
       'name': 'ZHU',
       'type': 'artist',
       'uri': 'spotify:artist:28j8lBWDdDSHSSt5oPlsX2'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
      'DM',
   